In [1]:
!pip install hvac

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.9/144.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyhcl: filename=pyhcl-0.4.4-py3-none-any.whl size=50127 sha256=b60624768dcdd383e0982cb0fe5333a081d609728e971e9a522c9557e481d644
  Stored in directory: /root/.cache/pip/wheels/6c/ad/33/e11e917cf04cb1533cab6e7aaa8cee93c950aa82c32398b83e
Successfully built pyhcl


In [17]:
import hvac
import sys
import time

import subprocess
import threading

def create_vault_client():
    # Set up the Vault client
    client = hvac.Client(url='http://localhost:8200')

    # Authenticate with the root token provided when starting the Vault server in development mode
    client.token = 'dev-only-token' #THIS IS FOR TESTING ONLY, WE WOULD NEVER STORE HERE

    return client

def run_command():
    command = "vault server -dev -dev-root-token-id=dev-only-token" #TESTING ONLY THIS IS IN DEV MODE
    subprocess.run(command, shell=True)

def get_stored_token(client):
    # Set up the Vault client
    #client = hvac.Client(url='http://localhost:8200')

    # Authenticate with the root token provided when starting the Vault server in development mode
    #client.token = 'dev-only-token'

    # Retrieve the stored token from Vault
    try:
        response = client.secrets.kv.v2.read_secret_version(
            path='secret/data/token',raise_on_deleted_version=True
        )
        stored_token = response['data']['data']['value']
        return stored_token
    except Exception as e:
        print("Error retrieving stored token:", e)
        return None

run_vault = threading.Thread(target=run_command)
run_vault.start()
time.sleep(5)

# Set up the Vault client
client = hvac.Client(url='http://localhost:8200')

# Authenticate with the root token provided when starting the Vault server in development mode
client.token = 'dev-only-token'

# Store the token in the Vault
create_response = client.secrets.kv.v2.create_or_update_secret(
    path='secret/data/token',
    secret=dict(value='!Q#E%T&U8i6y4r2w')
)

# For testing
"""
stored_token = get_stored_token()
if stored_token:
    print("Stored token:", stored_token)
else:
    print("Failed to retrieve stored token")
"""


ConnectionError: ignored